1. data 로드 분석후 전처리 진행
2. 분석된 내용을 기반으로 문제를 선택하고 학습후 결과를 계산
3. data를 3종으로 나누고 최종 test 데이터의 점수가 높게 나오도록 자유롭게 모델을 튜닝하여 결과를 만드시오

In [22]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from string import punctuation
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('spam.csv',encoding='latin1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [7]:
t_d = pd.DataFrame()
t_d['X'] = data['v2']
t_d['Y'] = data['v1']
t_d

,X,Y
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [9]:
t_d.Y.replace('ham',1,inplace=True)
t_d.Y.replace('spam',0,inplace=True)
t_d

,X,Y
0,"Go until jurong point, crazy.. Available only ...",1
1,Ok lar... Joking wif u oni...,1
2,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,U dun say so early hor... U c already then say...,1
4,"Nah I don't think he goes to usf, he lives aro...",1
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,0
5568,Will Ì_ b going to esplanade fr home?,1
5569,"Pity, * was in mood for that. So...any other s...",1
5570,The guy did some bitching but I acted like i'd...,1


In [10]:
t_d.isna().sum()

X    0
Y    0
dtype: int64

In [19]:
st = set(stopwords.words('english'))
p_ck = list(punctuation)
st.update(p_ck)

In [14]:
def 웹문서_처리(t):
    su = BeautifulSoup(t,'html.parser')
    return su.get_text()
def 정규표현_정리(t):
    return re.sub('\[[^]*\]','',t)
def 불필요_정리(t):
    return re.sub('http\S+','',t)
def 불용어_처리(t):
    f_l = []
    for i in t.split():
        if i.strip().lower() not in st and i.strip().lower().isalpha():
            f_l.append(i.strip().lower())
    return ' '.join(f_l)
def 모두_정리(t):
    t = 웹문서_처리(t)
    t = 불필요_정리(t)
    t = 불용어_처리(t)
    return t
t_d['X'] = t_d['X'].apply(모두_정리)

In [15]:
t_d

,X,Y
0,go jurong available bugis n great world la e c...,1
1,ok joking wif u,1
2,free entry wkly comp win fa cup final tkts may...,0
3,u dun say early u c already,1
4,nah think goes lives around though,1
...,...,...
5567,time tried contact u pound claim call per,0
5568,b going esplanade fr,1
5569,mood,1
5570,guy bitching acted like interested buying some...,1


In [20]:
t_d.isna().sum()

X    0
Y    0
dtype: int64